# Data Description

This dataset contains information about used cars, including their specifications, sale details, and market values. Below is a description of each column:

- **year**: The manufacturing year of the car.
- **make**: The brand or manufacturer of the car (e.g., Toyota, Ford).
- **model**: The specific model of the car (e.g., Camry, Mustang).
- **trim**: The trim level or variant of the car model, indicating additional features or specifications.
- **body**: The body type of the car (e.g., Sedan, SUV, Truck).
- **transmission**: The type of transmission (e.g., Automatic, Manual).
- **vin**: The Vehicle Identification Number, a unique code for each car.
- **state**: The state where the car was sold or located.
- **condition**: The condition of the car (e.g., New, Used, Excellent, Good).
- **odometer**: The mileage of the car, indicating how many miles it has been driven.
- **color**: The exterior color of the car.
- **interior**: The interior color or material of the car.
- **seller**: The name or type of seller (e.g., Dealer, Private).
- **mmr**: The Manheim Market Report value, an estimate of the car's wholesale price.
- **sellingprice**: The actual price at which the car was sold.
- **saledate**: The date when the car was sold.

# Data Preparation

## 1. Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()


In [ ]:
df = pd.read_csv(r"car_prices.csv")
df.head()

## 2. Data Overview

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.count()

In [ ]:
df.shape

In [ ]:
df.columns = df.columns.str.strip().str.lower()
df.columns


## 3. Data Cleaning

### Handling Missing Values

In [ ]:
plt.figure(figsize=(12, 6))

sns.heatmap(df.isna())

plt.show()

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum()/df.shape[0]*100

In [ ]:
def fill_na_with_mode(df, columns):
    """
    Fill missing values (NaN) in specified columns with the mode (most frequent value).

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    columns (list): A list of column names to fill missing values in.

    Returns:
    pd.DataFrame: The DataFrame with missing values filled in the specified columns.
    """


    for col in columns:
        mode_value = df[col].mode()[0]
        df[col].fillna(mode_value, inplace=True)

    return df

columns_to_fill = ['make', 'model', 'trim', 'body', 'transmission']
df = fill_na_with_mode(df, columns_to_fill)

In [ ]:
df.isna().mean()*100

In [ ]:
columns_to_drop = ['vin']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.columns

In [ ]:
df = df.dropna(subset=['sellingprice', 'saledate', 'mmr', 'make', 'color', 'odometer', 'condition', 'interior'])

In [ ]:
df.isna().mean() * 100

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.head()

### Checking for Duplicate Rows

In [ ]:
df.duplicated().sum()

## 4. Feature Engineering

### Handling Date Data

In [ ]:
df.head()

df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce', utc=True)

sales_per_day = df['saledate'].dt.date.value_counts()



top_10_sales_days = sales_per_day.head(10)



print("Top 10 sales days:")

display(top_10_sales_days)

In [ ]:
# Convert the 'saledate' column to datetime format with UTC timezone
df['saledate'] = pd.to_datetime(df['saledate'], utc=True)

# Extract the year from the 'saledate' column and store it in a new column 'sale_year'
df['sale_year'] = df['saledate'].dt.year

# Extract the month from the 'saledate' column and store it in a new column 'sale_month'
df['sale_month'] = df['saledate'].dt.month

# Extract the day from the 'saledate' column and store it in a new column 'sale_day'
df['sale_day'] = df['saledate'].dt.day

# Extract the day of the week (0=Monday, 6=Sunday) from the 'saledate' column and store it in a new column 'sale_dayofweek'
df['sale_dayofweek'] = df['saledate'].dt.dayofweek

# Display the data types of all columns in the DataFrame to verify the changes
df.dtypes

## 5. Standardizing the condition Value

In [ ]:
unique_conditions = df['condition'].unique()

unique_conditions

In [ ]:
sns.lineplot(data=df, x='condition', y='sellingprice', color='red')
plt.title('Selling Price by Condition', fontsize=16)
plt.xlabel('Condition', fontsize=12)
plt.ylabel('Selling Price ($)', fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.grid(False)

plt.tight_layout()
plt.show()


#### Selling Price by Condition

This visualization represents the **selling price** of cars categorized by their **condition**. The y-axis shows the selling price, while the x-axis represents different conditions .

In [ ]:
def unify_condition(value):
    """
    Unifies the condition value to a standardized scale.

    This function takes a numeric value representing a car's condition and converts it
    to a standardized scale (1-5). If the value is already within the 1-5 range, it is
    returned as is. If the value is between 10 and 50, it is divided by 10 and rounded
    to the nearest integer. For values outside these ranges, the function returns None.

    """
    if 1 <= value <= 5:
        return value
    elif 10 <= value <= 50:
        return round(value / 10)
    else:
        return None

In [ ]:
df['condition'] = df['condition'].apply(unify_condition)

df['condition'].unique()

In [ ]:
df['condition'].isnull().sum()/(df.shape[0])*100

In [ ]:
df['condition'] = df['condition'].astype(int)

record_counts = df['condition'].value_counts().reset_index()

record_counts

#### **Renaming** States

In [ ]:
state_names = {

    'ca': 'California', 'tx': 'Texas', 'pa': 'Pennsylvania', 'mn': 'Minnesota',

    'az': 'Arizona', 'wi': 'Wisconsin', 'tn': 'Tennessee', 'md': 'Maryland',

    'fl': 'Florida', 'ne': 'Nebraska', 'nj': 'New Jersey', 'nv': 'Nevada',

    'oh': 'Ohio', 'mi': 'Michigan', 'ga': 'Georgia', 'va': 'Virginia',

    'sc': 'South Carolina', 'nc': 'North Carolina', 'in': 'Indiana',

    'il': 'Illinois', 'co': 'Colorado', 'ut': 'Utah', 'mo': 'Missouri',

    'ny': 'New York', 'ma': 'Massachusetts', 'pr': 'Puerto Rico', 'or': 'Oregon',

    'la': 'Louisiana', 'wa': 'Washington', 'hi': 'Hawaii', 'qc': 'Quebec',

    'ab': 'Alberta', 'on': 'Ontario', 'ok': 'Oklahoma', 'ms': 'Mississippi',

    'nm': 'New Mexico', 'al': 'Alabama', 'ns': 'Nova Scotia'

}

df['state'] = df['state'].map(state_names)

df.head()

## 6. Outlier Handling

Removing Illogical Values from sellingpric

In [ ]:
df['Price_diff']=df['sellingprice'] - df['mmr']

In [ ]:
def outliers_handling(x, p1=0.25, p2=0.75):
    """
    Handles outliers in a numeric dataset using the Interquartile Range (IQR) method.

    This function calculates the first quartile (Q1) and the third quartile (Q3) of the dataset.
    It then computes the Interquartile Range (IQR) and uses it to determine the lower and upper
    limits for identifying outliers. Any value below the lower limit or above the upper limit
    is considered an outlier and is filtered out.

    """
    # Calculate the first quartile (Q1) and third quartile (Q3)
    q1 = x.quantile(p1)
    q3 = x.quantile(p2)

    # Calculate the Interquartile Range (IQR)
    iqr = q3 - q1

    # Define the lower and upper limits for outliers
    min_limit = q1 - 1.5 * iqr
    max_limit = q3 + 1.5 * iqr

    # Filter out the outliers and return the cleaned data
    return x[(x > min_limit) & (x < max_limit)]

In [ ]:
for col in ['odometer','mmr','sellingprice']:

    df[col] = outliers_handling(df[col])

In [ ]:
df['Price_diff'] = outliers_handling(df['Price_diff'])

In [ ]:
df.isna().sum()/(df.shape[0])*100

In [ ]:
df.isna().sum()

Removing the illogical differences between MMR and selling price in some rows

In [ ]:
df['sell_price']=df['Price_diff']+ df['mmr']

Handling Missing and Inconsistent Values in Categorical Columns

In [ ]:
Categorical_col = df.select_dtypes(include= 'object').columns

In [ ]:
df[Categorical_col] = df[Categorical_col].apply(lambda col: col.str.upper())

In [ ]:
df['seller'].unique()

In [ ]:
df['seller'] = df['seller'].str.lower().str.strip()

df['seller'] = df['seller'].str.replace('-', ' ', regex=False)

In [ ]:
df['seller'].value_counts()

In [ ]:
df['seller'].unique()

In [ ]:
columns_with_weird_values = ['color', 'interior']

for column in columns_with_weird_values:
    """
    Analyzes columns with weird values (e.g., '—') and calculates their percentage in the dataset.

    This function iterates over specified columns and counts the number of rows containing
    the weird value '—'. It then calculates the percentage of these rows relative to the
    total number of rows in the dataset.

    """
    # Count the number of rows with the weird value '—'
    weird_rows_count = df[df[column] == '—'].shape[0]

    # Get the total number of rows in the DataFrame
    total_rows = df.shape[0]

    # Calculate the percentage of rows with the weird value
    percentage_weird_rows = (weird_rows_count / total_rows) * 100


    print(f'Column: {column}')
    print(f'Weird rows count: {weird_rows_count}')
    print(f'Percentage of weird rows: {percentage_weird_rows:.2f}%\n')

In [ ]:
for col in ['color', 'interior']:
    """
    Replaces weird values (e.g., '—') in specified columns with the most common value (mode).

    This function iterates over the specified columns, identifies the most common value (mode),
    and replaces all occurrences of the weird value '—' with this mode. It then calculates and
    prints the percentage of rows that still contain the weird value after replacement.

    """
    # Find the most common value (mode) in the column
    most_common_value = df[col].mode()[0]

    # Replace the weird value '—' with the most common value
    df[col] = df[col].replace('—', most_common_value)

    # Count the number of rows that still contain the weird value '—'
    weird_rows_count = df[df[col] == '—'].shape[0]

    # Calculate the percentage of rows that still contain the weird value
    percentage_weird_rows = (weird_rows_count / df.shape[0]) * 100

    # Print the percentage of rows with the weird value after replacement
    print(f"Percentage of rows with weird '{col}' values after replacement: {percentage_weird_rows:.2f}%")

In [ ]:
df.isnull().mean() * 100

In [ ]:
df = df.dropna(subset=['sell_price'])

df = df.dropna(subset=['odometer'])

df = df.drop(columns=['Price_diff'])

df = df.drop(columns=['sellingprice'])

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
# Get the list of columns in the DataFrame
cols = df.columns.tolist()

# Remove 'sell_price' from the list
cols.remove('sell_price')

# Insert 'sell_price' after the 'mmr' column
cols.insert(cols.index('mmr') + 1, 'sell_price')

# Reorder the DataFrame columns based on the updated list
df = df[cols]

In [ ]:
df.dtypes

In [ ]:
df.isnull().mean() * 100

#### to CSV for ML

In [ ]:
df.to_csv(r"cleaned_car_prices.csv", index=False)